# Demo of Coregionalisation for Calibration

We can calibrate one sensor with another, using coregionalisation.

We need to estimate the vector, $W$, to produce the rank-one matrix $W W^\top$.

This is done below by looking at the RMSE, however it would be better to use the log-likelihood.

In particular we should use a laplace approximation, or the CCD approximation to get the uncertainty over this coregionalisation. We can then make predictions with the uncertainty of the coregionalisation.

The idea would be that additional observations would reduce this uncertainty.

We could also in the future look at the coregionalisation such that this varies over time...

In [1]:
#import GPy
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def sim(x):
    if x[1]==0:
        v = x[0] #pollution rising linearly with time in location 0
    if x[1]==1:
        v = np.sin(x[0]) #pollution rising as square of time in loc 1
    scales = [1,0.5,0.3]
    return scales[int(x[2])]*v
    
X = np.zeros([0,3])

N = 6 #number of time points
#time, space, region
#fixed station 0
X = np.r_[X,np.c_[np.arange(N)[:,None],np.full(N,0.0)[:,None],np.full(N,0)[:,None]]]
#mobile
X = np.r_[X,np.c_[np.arange(N)[:,None],np.full(N,1.0)[:,None],np.full(N,1)[:,None]]]
X[-3:,1] = 0
#fixed station 1
X = np.r_[X,np.c_[np.arange(N)[:,None],np.full(N,1.0)[:,None],np.full(N,2)[:,None]]]


Y = np.empty([len(X),1])
for i,x in enumerate(X):
    Y[i,0] = sim(x)
#latent real values
#X = np.r_[X,np.c_[np.arange(4)[:,None],np.full(4,0.0)[:,None],np.full(4,3)[:,None]]]
#X = np.r_[X,np.c_[np.arange(4)[:,None],np.full(4,1.0)[:,None],np.full(4,3)[:,None]]]



def k(X,X2,W,l_time=2.0,l_dist=0.1):
    coregmat = W.T @ W
    k_time = np.exp(-(X[0]-X2[0])**2/(2*l_time))
    k_dist = np.exp(-(X[1]-X2[1])**2/(2*l_dist))
    k_coreg = coregmat[int(X[2]),int(X2[2])]
    return k_time * k_dist * k_coreg

def calcK(X,W = np.array([[1,1,1]])):
    K = np.empty([len(X),len(X)])
    for i,x in enumerate(X):
        for j,x2 in enumerate(X):
            K[i,j] = k(x,x2,W)
    return K

def calckstar(xtest,X,W = np.array([[1,1,1]])):
    kstar = np.empty([len(X),1])
    for i in range(len(X)):
        kstar[i,:]=k(xtest,X[i,:],W)
    return kstar

leastrmse = np.inf
leastWs = None
for W0 in np.arange(0.4,6.01,0.6):
    for W1 in np.arange(0.4,6.01,0.6):
        for W2 in np.arange(0,6.01,0.6):
            W = np.array([[W0,W1,W2]])
            rmse = []
            for dropi in range(len(X)):
                Xdrop = np.delete(X,dropi,0)
                Ydrop = np.delete(Y,dropi,0)
                K = calcK(Xdrop,W)
                xtest = X[dropi,:]
                kstar = calckstar(xtest,Xdrop,W)    
                #kstarstar = k(xtest,xtest)
                Kinv = np.linalg.inv(K+np.eye(len(K)))
                predy = kstar.T@Kinv@Ydrop
                rmse.append((predy-Y[dropi])**2)
                #kstarstar - kstar.T@Kinv@kstar
            rmse = np.sqrt(np.mean(np.array(rmse)))
            print(W,rmse)
            if rmse<leastrmse:
                leastrmse = rmse
                leastWs = W

[[0.4 0.4 0. ]] 1.3764795588348517
[[0.4 0.4 0.6]] 1.373167166863018
[[0.4 0.4 1.2]] 1.3721625955320913
[[0.4 0.4 1.8]] 1.372068758910044
[[0.4 0.4 2.4]] 1.3721657867200354
[[0.4 0.4 3. ]] 1.3722787487050783
[[0.4 0.4 3.6]] 1.3723748413146877
[[0.4 0.4 4.2]] 1.3724520597301086
[[0.4 0.4 4.8]] 1.3725141228591131
[[0.4 0.4 5.4]] 1.3725649315922699
[[0.4 0.4 6. ]] 1.3726075591225144
[[0.4 1.  0. ]] 1.4602570291596884
[[0.4 1.  0.6]] 1.4566303705739163
[[0.4 1.  1.2]] 1.4555723428091527
[[0.4 1.  1.8]] 1.456133495709564
[[0.4 1.  2.4]] 1.4569073460577142
[[0.4 1.  3. ]] 1.4575399965785205
[[0.4 1.  3.6]] 1.4580144828821204
[[0.4 1.  4.2]] 1.458370112599097
[[0.4 1.  4.8]] 1.4586435501421604
[[0.4 1.  5.4]] 1.4588610609587795
[[0.4 1.  6. ]] 1.459040384140658
[[0.4 1.6 0. ]] 1.5916805274140688
[[0.4 1.6 0.6]] 1.5884418866553014
[[0.4 1.6 1.2]] 1.5867649047203485
[[0.4 1.6 1.8]] 1.5870161826471487
[[0.4 1.6 2.4]] 1.5879366527480359
[[0.4 1.6 3. ]] 1.588892115272362
[[0.4 1.6 3.6]] 1.58970938

[[1.6 1.  3.6]] 0.5916198607870299
[[1.6 1.  4.2]] 0.5924957121042066
[[1.6 1.  4.8]] 0.593168667642596
[[1.6 1.  5.4]] 0.5937035438994358
[[1.6 1.  6. ]] 0.5941441154443419
[[1.6 1.6 0. ]] 0.9391371266680876
[[1.6 1.6 0.6]] 0.9337126070958894
[[1.6 1.6 1.2]] 0.9308680487553521
[[1.6 1.6 1.8]] 0.9312916525389988
[[1.6 1.6 2.4]] 0.9328563064021093
[[1.6 1.6 3. ]] 0.9344812244692149
[[1.6 1.6 3.6]] 0.9358707714015254
[[1.6 1.6 4.2]] 0.9370002263844395
[[1.6 1.6 4.8]] 0.9379144182211018
[[1.6 1.6 5.4]] 0.9386653451202075
[[1.6 1.6 6. ]] 0.9392964003521415
[[1.6 2.2 0. ]] 1.2798739808234163
[[1.6 2.2 0.6]] 1.2761793422991476
[[1.6 2.2 1.2]] 1.2736384027597283
[[1.6 2.2 1.8]] 1.273254751969591
[[1.6 2.2 2.4]] 1.274080413105873
[[1.6 2.2 3. ]] 1.2753047360017253
[[1.6 2.2 3.6]] 1.2765507087579573
[[1.6 2.2 4.2]] 1.277684571394678
[[1.6 2.2 4.8]] 1.2786775817521447
[[1.6 2.2 5.4]] 1.27954009968065
[[1.6 2.2 6. ]] 1.2802938745507417
[[1.6 2.8 0. ]] 1.5513127538561076
[[1.6 2.8 0.6]] 1.54846712

[[2.8 2.2 0. ]] 0.6835141120626611
[[2.8 2.2 0.6]] 0.6766211052234056
[[2.8 2.2 1.2]] 0.6718145745479694
[[2.8 2.2 1.8]] 0.6710732954521079
[[2.8 2.2 2.4]] 0.672623965476652
[[2.8 2.2 3. ]] 0.6749270299409196
[[2.8 2.2 3.6]] 0.6772671433234316
[[2.8 2.2 4.2]] 0.6793924567253464
[[2.8 2.2 4.8]] 0.6812502211060203
[[2.8 2.2 5.4]] 0.6828611359660203
[[2.8 2.2 6. ]] 0.6842668928471632
[[2.8 2.8 0. ]] 0.9841362536332353
[[2.8 2.8 0.6]] 0.9797168794026799
[[2.8 2.8 1.2]] 0.9762826448983052
[[2.8 2.8 1.8]] 0.9750375118990326
[[2.8 2.8 2.4]] 0.9754276090321415
[[2.8 2.8 3. ]] 0.9766486412407935
[[2.8 2.8 3.6]] 0.9781779719664283
[[2.8 2.8 4.2]] 0.9797483538500835
[[2.8 2.8 4.8]] 0.9812431600569199
[[2.8 2.8 5.4]] 0.9826228592863464
[[2.8 2.8 6. ]] 0.9838840076369039
[[2.8 3.4 0. ]] 1.252832872519901
[[2.8 3.4 0.6]] 1.2495737926162427
[[2.8 3.4 1.2]] 1.246991243245674
[[2.8 3.4 1.8]] 1.2456597512436025
[[2.8 3.4 2.4]] 1.245476085926457
[[2.8 3.4 3. ]] 1.246036388635673
[[2.8 3.4 3.6]] 1.2469899

[[4.  2.8 3.6]] 0.5312516746112584
[[4.  2.8 4.2]] 0.5341317538840102
[[4.  2.8 4.8]] 0.536863327774838
[[4.  2.8 5.4]] 0.5393761047283465
[[4.  2.8 6. ]] 0.5416659875747869
[[4.  3.4 0. ]] 0.7929475695043152
[[4.  3.4 0.6]] 0.7878150577318906
[[4.  3.4 1.2]] 0.783707029931195
[[4.  3.4 1.8]] 0.7815786688104086
[[4.  3.4 2.4]] 0.7812795556639783
[[4.  3.4 3. ]] 0.7821669320102117
[[4.  3.4 3.6]] 0.7836800179269996
[[4.  3.4 4.2]] 0.7854597416376843
[[4.  3.4 4.8]] 0.7873053807828703
[[4.  3.4 5.4]] 0.789116015067528
[[4.  3.4 6. ]] 0.7908479901046651
[[4. 4. 0.]] 1.026911013233807
[[4.  4.  0.6]] 1.0231627603955322
[[4.  4.  1.2]] 1.0202555577768928
[[4.  4.  1.8]] 1.0183872049715563
[[4.  4.  2.4]] 1.0176897430401106
[[4. 4. 3.]] 1.0179011805585672
[[4.  4.  3.6]] 1.0186908491984699
[[4.  4.  4.2]] 1.0198049373932712
[[4.  4.  4.8]] 1.0210774479144462
[[4.  4.  5.4]] 1.0224089042953775
[[4. 4. 6.]] 1.0237439662901167
[[4.  4.6 0. ]] 1.2407668226281277
[[4.  4.6 0.6]] 1.237805603211558

[[5.2 4.  0.6]] 0.6615294049730255
[[5.2 4.  1.2]] 0.6570189587532663
[[5.2 4.  1.8]] 0.6541099856792502
[[5.2 4.  2.4]] 0.6530213507202933
[[5.2 4.  3. ]] 0.6533489908088232
[[5.2 4.  3.6]] 0.6545766670062841
[[5.2 4.  4.2]] 0.6563070695332627
[[5.2 4.  4.8]] 0.6582805213750356
[[5.2 4.  5.4]] 0.6603418384179973
[[5.2 4.  6. ]] 0.6624050864084495
[[5.2 4.6 0. ]] 0.8698348854780829
[[5.2 4.6 0.6]] 0.8656190659084172
[[5.2 4.6 1.2]] 0.8625454395698244
[[5.2 4.6 1.8]] 0.8602113299330318
[[5.2 4.6 2.4]] 0.8589757964503185
[[5.2 4.6 3. ]] 0.8587470398213283
[[5.2 4.6 3.6]] 0.859245285017967
[[5.2 4.6 4.2]] 0.8602128931909859
[[5.2 4.6 4.8]] 0.8614599105240345
[[5.2 4.6 5.4]] 0.8628591673620469
[[5.2 4.6 6. ]] 0.8643306024473462
[[5.2 5.2 0. ]] 1.0582924437885832
[[5.2 5.2 0.6]] 1.0549761445904482
[[5.2 5.2 1.2]] 1.052788790968044
[[5.2 5.2 1.8]] 1.0508609359569283
[[5.2 5.2 2.4]] 1.0496082630330335
[[5.2 5.2 3. ]] 1.0491003322514383
[[5.2 5.2 3.6]] 1.0491869068174813
[[5.2 5.2 4.2]] 1.0496

In [ ]:
leastWs

In [ ]:
np.array([1,0.5,0.3])*5.8

In [ ]:
plt.matshow(K)